In [ ]:
!pip install -q datasets 
!pip install -q chromadb
!pip install -q langchain
!pip install -q wikipedia
!pip install -q langchain-community
!pip install -q unstructured
!pip install -q nltk
!pip install -q stanza
!pip install -q googletrans==3.1.0a0
!pip install -q pyarabic
!pip install -q pydantic


In [2]:
#!pip install pytesseract pillow pdf2image
#!sudo apt install tesseract-ocr tesseract-ocr-ara  # Arabic language support for Tesseract

In [3]:
#!apt-get install poppler-utils

In [4]:
#2
#packages
#import
import numpy as np
import pandas as pd
from datasets import load_dataset


In [5]:
#from pdf2image import convert_from_path
#import pytesseract

##Step 1: Data Preprocessing


In [7]:
# Open the .txt file and read its content as a string
with open('/content/law.txt', 'r') as file:
    Raw_data = file.read()

# Print the content of the file
print(Raw_data)


1 
  ﻗﺎﻧﻭﻥ ﺭﻗﻡ12   ﻟﺳﻧﺔ 2003 
ﺑﺈﺻﺩﺍﺭ ﻗﺎﻧﻭﻥ ﺍﻟﻌﻣﻝ  
 
 
ﺑﺎﺳﻡ ﺍﻟﺷﻌﺏ  
ﺭﺋﻳﺱ ﺍﻟﺟﻣﻬﻭﺭﻳﺔ  
ﺑﻌﺩ ﺍﻹﻁﻼﻉ ﻋﻠﻰ ﺍﻟﺩﺳﺗﻭﺭ  
 
ﻗﺭﺭ ﻣﺟﻠﺱ ﺍﻟﺷﻌﺏ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻷﺗﻲ ﻧﺻﻪ  
 
(ﺍﻟﻣﺎﺩﺓ ﺍﻷﻭﻟﻰ)  
ﻳﻌﻣﻝ ﺑﺄﺣﻛﺎﻡ ﻗﺎﻧﻭﻥ ﺍﻟﻌﻣﻝ ﺍﻟﻣﺭﺍﻓﻕ .
 
 
(ﺍﻟﻣﺎﺩﺓ ﺍﻟﺛﺎﻧﻳﺔ)  
ﺗﻅﻝ ﺍﻷﺣﻛﺎﻡ ﺍﻟﻭﺍﺭﺩﺓ ﺑﺎﻟﺗﺷﺭﻳﻌﺎﺕ ﺍﻟﺧﺎﺻﺔ ﺑﺑﻌﺽ ﻓﺋﺎﺕ ﺍﻟﻌﻣﺎﻝ ﺳﺎﺭﻳﺔ ﻭﺫﻟﻙ ﺇﻟﻰ ﺃﻥ ﻳﺗﻡ ﺇﺑﺭﺍﻡ ﻭﻧﻔﺎﺫ ﺍﻻﺗﻔﺎﻗﻳﺎﺕ ﺍﻟﺟﻣﺎﻋﻳﺔ ﺑﺷﺄﻧﻬﺎ ﻭﻓﻘﺎ 
ﻷﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ .
ﻭﺗﻌﺗﺑﺭ ﺍﻟﻣﺯﺍﻳﺎ ﺍﻟﻭﺍﺭﺩﺓ ﺑﺗﻠﻙ ﺍﻟﺗﺷﺭﻳﻌﺎﺕ ﺍﻟﺣﺩ ﺍﻷﺩﻧﻰ ﺍﻟﺫﻱ ﻳﺗﻡ ﺍﻟﺗﻔﺎﻭﺽ ﻋﻠﻰ ﺃﺳﺎﺳﻪ .
 
(ﺍﻟﻣﺎﺩﺓ ﺍﻟﺛﺎﻟﺛﺔ)  
ﻳﺳﺗﺣﻕ ﺍﻟﻌﺎﻣﻠﻭﻥ ﺍﻟﺫﻳﻥ ﺗﺳﺭﻯ ﻓﻲ ﺷﺄﻧﻬﻡ ﺃﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ ﻋﻼﻭﺓ ﺳﻧﻭﻳﺔ ﺩﻭﺭﻳﺔ ﻓﻲ) ﺗﺎﺭﻳﺦ ﺍﺳﺗﺣﻘﺎﻗﻬﺎ ﻻ ﺗﻘﻝ ﻋﻥ 7  ﻣﻥ ﺍﻷﺟﺭ%(  
ﺍﻷﺳﺎﺳﻲ ﺍﻟﺫﻱ ﺗﺣﺳﺏ ﻋﻠﻰ ﺃﺳﺎﺳﻪ ﺍﺷﺗﺭﺍﻛﺎﺕ ﺍﻟﺗﺄﻣﻳﻧﺎﺕ ﺍﻻﺟﺗﻣﺎﻋﻳﺔ، ﻭﺫﻟﻙ ﺣﺗﻰ ﻳﺻﺩﺭ ﺍﻟﻣﺟﻠﺱ ﺍﻟﻘﻭﻣﻲ ﻟﻸﺟﻭﺭ ﺍﻟﻘﺭﺍﺭﺍﺕ ﺍﻟﻣﻧﻅﻣﺔ ﻟﻬﺫﻩ 
ﺍﻟﻌﻼﻭﺓ .
(ﺍﻟﻣﺎﺩﺓ ﺍﻟﺭﺍﺑﻌﺔ)  
ﻻ ﺗﺧﻝ ﺃﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ ﺑﺣﻘﻭﻕ ﺍﻟﻌﻣﺎﻝ ﺍﻟﺳﺎﺑﻕ ﻟﻬﻡ ﺍﻟﺣﺻﻭﻝ ﻋﻠﻳﻬﺎ ﻣﻥ ﺃﺟﻭﺭ ﻭﻣﺯﺍﻳﺎ ﻣﺳﺗﻣﺩﺓ ﻣﻥ ﺃﺣﻛﺎﻡ ﺍﻟﻘﻭﺍﻧﻳﻥ ﻭﺍﻟﻠﻭﺍﺋﺢ ﻭﺍﻟﻧﻅﻡ  
. ﻭﺍﻻﺗﻔﺎﻗﻳﺎﺕ ﻭﺍﻟﻘﺭﺍﺭﺍﺕ ﺍﻟﺩﺍﺧﻠﻳﺔ ﺍﻟﺳﺎﺑﻘﺔ ﻋﻠﻰ ﺍﻟﻌﻣﻝ ﺑﺄﺣﻛﺎﻣﻪ  
(ﺍﻟﻣﺎﺩﺓ ﺍﻟﺧﺎﻣﺳﺔ)  
ﺗﺅﻭﻝ ﺇﻟﻰ ﻭﺯﺍﺭﺓ ﺍﻟﻘﻭﻯ ﺍﻟﻌﺎﻣﻠﺔ ﻭﺍﻟﻬﺟﺭﺓ ﺟﻣﻳﻊ ﺍﻟﻣﺑﺎﻟﻎ ﺍﻟﻣﺣﻛﻭﻡ ﺑﻬﺎ ﻋﻥ ﻣﺧﺎﻟﻔﺔ ﺃﺣ

In [8]:
#9
!pip install pyarabic
from nltk.corpus import stopwords
from textblob import TextBlob
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}


import pyarabic.araby as araby
def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)

def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    # return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])
    return " ".join([w for w in words if not w in stops and len(w) >= 2])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [9]:
#import stanza

# Download Arabic models for Stanza
#stanza.download('ar')
#nlp = stanza.Pipeline('ar')

In [10]:
# Function to lemmatize a single string
#def lemmatize_text(text):
#    doc = nlp(text)
#    lemmas = [word.lemma for sentence in doc.sentences for word in sentence.words]
#    return ' '.join(lemmas)

In [11]:
#14
!python -m textblob.download_corpora
#nlp = stanza.Pipeline('ar', use_gpu=False)  # fix lemmatization error

def clean_text(text):

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)

    ## Translate english words

    ## Remove stop words
    text = remove_stop_words(text)

    ##Lemmatizing
    #text = lemmatize_text(text)

    ## Remove Tashkeel
    text = normalizeArabic(text)

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    return text

# Assuming docs_text is your existing list
docs_text = pd.Series(Raw_data)

# Now you can use the map function
cleaned_docs = docs_text.map(lambda x: clean_text(x))

# If you need it back as a list:
cleaned_docs = cleaned_docs.tolist()


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [12]:
#16
cleaned_docs

['ﻗﺎﻧﻭﻥ ﺭﻗﻡ12 ﻟﺳﻧﺔ 2003 ﺑﺈﺻﺩﺍﺭ ﻗﺎﻧﻭﻥ ﺍﻟﻌﻣﻝ ﺑﺎﺳﻡ ﺍﻟﺷﻌﺏ ﺭﺋﻳﺱ ﺍﻟﺟﻣﻬﻭﺭﻳﺔ ﺑﻌﺩ ﺍﻹﻁﻼﻉ ﻋﻠﻰ ﺍﻟﺩﺳﺗﻭﺭ ﻗﺭﺭ ﻣﺟﻠﺱ ﺍﻟﺷﻌﺏ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻷﺗﻲ ﻧﺻﻪ ﺍﻟﻣﺎﺩﺓ ﺍﻷﻭﻟﻰ ﻳﻌﻣﻝ ﺑﺄﺣﻛﺎﻡ ﻗﺎﻧﻭﻥ ﺍﻟﻌﻣﻝ ﺍﻟﻣﺭﺍﻓﻕ ﺍﻟﻣﺎﺩﺓ ﺍﻟﺛﺎﻧﻳﺔ ﺗﻅﻝ ﺍﻷﺣﻛﺎﻡ ﺍﻟﻭﺍﺭﺩﺓ ﺑﺎﻟﺗﺷﺭﻳﻌﺎﺕ ﺍﻟﺧﺎﺻﺔ ﺑﺑﻌﺽ ﻓﺋﺎﺕ ﺍﻟﻌﻣﺎﻝ ﺳﺎﺭﻳﺔ ﻭﺫﻟﻙ ﺇﻟﻰ ﺃﻥ ﻳﺗﻡ ﺇﺑﺭﺍﻡ ﻭﻧﻔﺎﺫ ﺍﻻﺗﻔﺎﻗﻳﺎﺕ ﺍﻟﺟﻣﺎﻋﻳﺔ ﺑﺷﺄﻧﻬﺎ ﻭﻓﻘﺎ ﻷﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ ﻭﺗﻌﺗﺑﺭ ﺍﻟﻣﺯﺍﻳﺎ ﺍﻟﻭﺍﺭﺩﺓ ﺑﺗﻠﻙ ﺍﻟﺗﺷﺭﻳﻌﺎﺕ ﺍﻟﺣﺩ ﺍﻷﺩﻧﻰ ﺍﻟﺫﻱ ﻳﺗﻡ ﺍﻟﺗﻔﺎﻭﺽ ﻋﻠﻰ ﺃﺳﺎﺳﻪ ﺍﻟﻣﺎﺩﺓ ﺍﻟﺛﺎﻟﺛﺔ ﻳﺳﺗﺣﻕ ﺍﻟﻌﺎﻣﻠﻭﻥ ﺍﻟﺫﻳﻥ ﺗﺳﺭﻯ ﻓﻲ ﺷﺄﻧﻬﻡ ﺃﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ ﻋﻼﻭﺓ ﺳﻧﻭﻳﺔ ﺩﻭﺭﻳﺔ ﻓﻲ ﺗﺎﺭﻳﺦ ﺍﺳﺗﺣﻘﺎﻗﻬﺎ ﺗﻘﻝ ﻋﻥ ﻣﻥ ﺍﻷﺟﺭ ﺍﻷﺳﺎﺳﻲ ﺍﻟﺫﻱ ﺗﺣﺳﺏ ﻋﻠﻰ ﺃﺳﺎﺳﻪ ﺍﺷﺗﺭﺍﻛﺎﺕ ﺍﻟﺗﺄﻣﻳﻧﺎﺕ ﺍﻻﺟﺗﻣﺎﻋﻳﺔ، ﻭﺫﻟﻙ ﺣﺗﻰ ﻳﺻﺩﺭ ﺍﻟﻣﺟﻠﺱ ﺍﻟﻘﻭﻣﻲ ﻟﻸﺟﻭﺭ ﺍﻟﻘﺭﺍﺭﺍﺕ ﺍﻟﻣﻧﻅﻣﺔ ﻟﻬﺫﻩ ﺍﻟﻌﻼﻭﺓ ﺍﻟﻣﺎﺩﺓ ﺍﻟﺭﺍﺑﻌﺔ ﺗﺧﻝ ﺃﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ ﺑﺣﻘﻭﻕ ﺍﻟﻌﻣﺎﻝ ﺍﻟﺳﺎﺑﻕ ﻟﻬﻡ ﺍﻟﺣﺻﻭﻝ ﻋﻠﻳﻬﺎ ﻣﻥ ﺃﺟﻭﺭ ﻭﻣﺯﺍﻳﺎ ﻣﺳﺗﻣﺩﺓ ﻣﻥ ﺃﺣﻛﺎﻡ ﺍﻟﻘﻭﺍﻧﻳﻥ ﻭﺍﻟﻠﻭﺍﺋﺢ ﻭﺍﻟﻧﻅﻡ ﻭﺍﻻﺗﻔﺎﻗﻳﺎﺕ ﻭﺍﻟﻘﺭﺍﺭﺍﺕ ﺍﻟﺩﺍﺧﻠﻳﺔ ﺍﻟﺳﺎﺑﻘﺔ ﻋﻠﻰ ﺍﻟﻌﻣﻝ ﺑﺄﺣﻛﺎﻣﻪ ﺍﻟﻣﺎﺩﺓ ﺍﻟﺧﺎﻣﺳﺔ ﺗﺅﻭﻝ ﺇﻟﻰ ﻭﺯﺍﺭﺓ ﺍﻟﻘﻭﻯ ﺍﻟﻌﺎﻣﻠﺔ ﻭﺍﻟﻬﺟﺭﺓ ﺟﻣﻳﻊ ﺍﻟﻣﺑﺎﻟﻎ ﺍﻟﻣﺣﻛﻭﻡ ﺑﻬﺎ ﻋﻥ ﻣﺧﺎﻟﻔﺔ ﺃﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ ﻭﻳﻛﻭﻥ ﺍﻟﺗﺻﺭﻑ ﻓﻳﻬﺎ ﻋﻠﻰ ﺍﻟﻭﺟﻪ ﺍﻵﺗﻲ ﺛﻠﺛﺎﻥ ﻳﺧﺻﺻﺎﻥ ﻟﻠﺻﺭﻑ ﻓﻲ ﺍﻷﻭ

In [13]:
# Convert the embeddings to a DataFrame
df_cleaned_docs = pd.DataFrame(cleaned_docs)

# Save the DataFrame to a CSV file
df_cleaned_docs.to_csv('cleaned_docs.csv', index=False)

##Step 2: Reading and Chunking
Use different Chunkcing techniques with meaningful variable name

In [14]:
#26
#!pip uninstall nltk langchain
#!pip install langchain==0.0.199

In [15]:
#27
#!pip install 'pydantic<2.0'

In [16]:
#28
#!pip install --upgrade "langchain>=0.0.200"

In [17]:
#29
sizes = [len(doc) for doc in cleaned_docs ]

if sizes:
    max_size = max(sizes)
    min_size = min(sizes)
    mean_size = sum(sizes) / len(sizes)
    print(f"Maximum size in cleaned_docs: {max_size} characters")
    print(f"Minimum size in cleaned_docs: {min_size} characters")
    print(f"Mean size in cleaned_docs: {mean_size:.2f} characters")
else:
    print("cleaned_docs is empty.")


Maximum size in cleaned_docs: 98356 characters
Minimum size in cleaned_docs: 98356 characters
Mean size in cleaned_docs: 98356.00 characters


In [18]:
#30
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap  = 100,
)

In [19]:
#31
new_chunks=[]

In [20]:
#32
for doc in cleaned_docs:
    size = len(doc)
    if size > 300:
        chunks = text_splitter.split_text(doc)
        new_chunks.extend(chunks)
    else:
        new_chunks.append(doc)

In [21]:
#33
print(len(new_chunks))

492


In [22]:
#34
sizes = [len(doc) for doc in new_chunks]
if sizes:
    max_size = max(sizes)
    min_size = min(sizes)
    mean_size = sum(sizes) / len(sizes)
    print(f"Maximum size in cleaned_docs: {max_size} characters")
    print(f"Minimum size in cleaned_docs: {min_size} characters")
    print(f"Mean size in cleaned_docs: {mean_size:.2f} characters")
else:
    print("cleaned_docs is empty.")


Maximum size in cleaned_docs: 299 characters
Minimum size in cleaned_docs: 207 characters
Mean size in cleaned_docs: 296.18 characters


In [23]:
#35
new_chunks[2]

'ﺍﻟﻣﺎﺩﺓ ﺍﻟﺛﺎﻟﺛﺔ ﻳﺳﺗﺣﻕ ﺍﻟﻌﺎﻣﻠﻭﻥ ﺍﻟﺫﻳﻥ ﺗﺳﺭﻯ ﻓﻲ ﺷﺄﻧﻬﻡ ﺃﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ ﻋﻼﻭﺓ ﺳﻧﻭﻳﺔ ﺩﻭﺭﻳﺔ ﻓﻲ ﺗﺎﺭﻳﺦ ﺍﺳﺗﺣﻘﺎﻗﻬﺎ ﺗﻘﻝ ﻋﻥ ﻣﻥ ﺍﻷﺟﺭ ﺍﻷﺳﺎﺳﻲ ﺍﻟﺫﻱ ﺗﺣﺳﺏ ﻋﻠﻰ ﺃﺳﺎﺳﻪ ﺍﺷﺗﺭﺍﻛﺎﺕ ﺍﻟﺗﺄﻣﻳﻧﺎﺕ ﺍﻻﺟﺗﻣﺎﻋﻳﺔ، ﻭﺫﻟﻙ ﺣﺗﻰ ﻳﺻﺩﺭ ﺍﻟﻣﺟﻠﺱ ﺍﻟﻘﻭﻣﻲ ﻟﻸﺟﻭﺭ ﺍﻟﻘﺭﺍﺭﺍﺕ ﺍﻟﻣﻧﻅﻣﺔ ﻟﻬﺫﻩ ﺍﻟﻌﻼﻭﺓ ﺍﻟﻣﺎﺩﺓ ﺍﻟﺭﺍﺑﻌﺔ ﺗﺧﻝ ﺃﺣﻛﺎﻡ ﺍﻟﻘﺎﻧﻭﻥ ﺍﻟﻣﺭﺍﻓﻕ ﺑﺣﻘﻭﻕ ﺍﻟﻌﻣﺎﻝ'

In [24]:
"""
import csv
# Save the text in a CSV file page by page with index
with open("chunks.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)

    # Loop through all pages
    for i, text in enumerate(new_chunks):
        # Write the page index and text
        writer.writerow([f"Page {i + 1}", text])

print("Text saved in extracted_text_by_page.csv")
"""

'\nimport csv\n# Save the text in a CSV file page by page with index\nwith open("chunks.csv", "w", newline=\'\', encoding="utf-8") as f:\n    writer = csv.writer(f)\n\n    # Loop through all pages\n    for i, text in enumerate(new_chunks):\n        # Write the page index and text\n        writer.writerow([f"Page {i + 1}", text])\n\nprint("Text saved in extracted_text_by_page.csv")\n'

## Step 3: Embedding
Extract Embedding using different models with meaningfull variable name

In [25]:
#!pip install sentence-transformers==2.3.1
#!pip install chromadb==0.4.22

In [26]:
# Step 1: Flatten and clean the data
#flattened_texts = [item[0].strip("[]'") for item in new_chunks]

In [27]:
#print (flattened_texts[:100])

In [29]:
from sentence_transformers import SentenceTransformer
import torch
model_id = 'intfloat/multilingual-e5-large'
dim = 512

# model_id = "asafaya/bert-large-arabic"
# dim = 1024

device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [30]:
embeddings_large = model.encode(new_chunks)

In [31]:
import chromadb

chroma_client = chromadb.PersistentClient(path="./chromadb-ar-docs6")

collection = chroma_client.create_collection(
    name="laww",
    metadata={"hnsw:space": "cosine"}
)

In [32]:
docs_ids = [
    str(i)
    for i in range( len(embeddings_large) )
]

In [33]:
"""
df_embeddings = pd.DataFrame(embeddings_large)

df_embeddings.to_csv('/content/embeddings_large.csv', index=False)

print("Embeddings saved to embeddings_large.csv")
"""

'\ndf_embeddings = pd.DataFrame(embeddings_large)\n\ndf_embeddings.to_csv(\'/content/embeddings_large.csv\', index=False)\n\nprint("Embeddings saved to embeddings_large.csv")\n'

In [34]:
"""
import pandas as pd

# Load the CSV file into a DataFrame
df_embeddings = pd.read_csv('/content/embeddings_large.csv')

# Display the first few rows to verify the contents
print(df_embeddings.head())

# Convert the DataFrame back to a list or numpy array (if needed)
embeddings_large = df_embeddings.values.tolist()

# Or, if you prefer a numpy array:
# embeddings_large = df_embeddings.to_numpy()

print("Embeddings extracted successfully")

"""

'\nimport pandas as pd\n\n# Load the CSV file into a DataFrame\ndf_embeddings = pd.read_csv(\'/content/embeddings_large.csv\')\n\n# Display the first few rows to verify the contents\nprint(df_embeddings.head())\n\n# Convert the DataFrame back to a list or numpy array (if needed)\nembeddings_large = df_embeddings.values.tolist()\n\n# Or, if you prefer a numpy array:\n# embeddings_large = df_embeddings.to_numpy()\n\nprint("Embeddings extracted successfully")\n\n'

In [35]:
#import pandas as pd
#new_chunks = pd.read_csv("/content/chunks.csv")

In [36]:
collection.add(
    documents=new_chunks,
    embeddings=embeddings_large,
    # metadatas=metadata,
    ids=docs_ids
)

#Prompt

In [37]:
#!pip install openpyxl
import pandas as pd

try:
  # Try reading the file with the 'utf-8' encoding first
  FAQs = pd.read_excel("//content/FAQs.xlsx", engine='openpyxl')
except UnicodeDecodeError:
  # If 'utf-8' fails, try 'latin-1' or 'ISO-8859-1'
  FAQs = pd.read_excel("//content/FAQs.xlsx", engine='openpyxl', encoding='latin-1')
  # or
  # FAQs = pd.read_excel("//content/FAQs.xlsx", engine='openpyxl', encoding='ISO-8859-1')

FAQs

,Question,Answer
0,س1 : ما معني العلاوة الدورية ؟,ج: العلاوة الدورية هي مبلغ نقدي ثابت يضاف الي ...
1,س : ما هي اشتراطات المشرع المصري في بعض العقود...,ج: يشترط المشروع المصري في بعض العقود أن تكون ...
2,التي لا يجوز تشغيل الأطفال فيها إذا قلت سنهم ع...,صدر في تحديد هذه الاعمال و المهن قـــرار رقم (...
3,س : ما حق صاحب العمل في رفضه قيام العامل بالاج...,ج: يبحث هذا الموضوع وإعمالاً لأحكام العمل الصا...
4,ما هي حالات الفصل التعسفي في قانون العمل المصر...,"وضع القانون رقم 12 لسنة 2003 ""قانون العمل المو..."
5,س : ما هو ميعاد استحقاق العمال للعلاوة الدورية؟,يستحق العاملون الذين تسري في شأنهم أحكام القان...
6,ما هو الميعاد الخاص ببطلان أي مخالصة عن حق من ...,يقع باطلاً كل شرط أو اتفاق يخالف أحكام هذا الق...
7,ما هو ميعاد قيد العاملين الجدد بمكتب القوي الع...,مع عدم الإخلال بأحكام القانون رقم 39 لسنة 1975...
8,ما هو ميعاد إرسال بيان مفصل عن العاملين الجدد ...,يلتزم صاحب العمل في المنشأة القائمة وقت تطبيق ...
9,ميعاد اعتراض وزارة القوي العاملة ؟,تقدم الجهات المشار إليها في المادتين 17 ، 18 م...


In [ ]:
questions = FAQs["Question"].tolist()
answers = FAQs["Answer"].tolist()

In [ ]:
questions[:5]

['س1 : ما معني العلاوة الدورية ؟',
 'س : ما هي اشتراطات المشرع المصري في بعض العقود المتصلة بقانون العمل ؟',
 'التي لا يجوز تشغيل الأطفال فيها إذا قلت سنهم عن سبع عشرة سنة ؟',
 'س : ما حق صاحب العمل في رفضه قيام العامل بالاجازة السنوية ؟',
 'ما هي حالات الفصل التعسفي في قانون العمل المصري ؟ و ما هي الاجراءات التي علي العامل اتباعها في حالة فصله فصلا تعسفيا ؟']

In [ ]:
answers[:5]

['ج: العلاوة الدورية هي مبلغ نقدي ثابت يضاف الي أجر العامل في موعد دوري غالباً ما يكون أول يناير من كل عام ، ويتكرر صرفها بمرور سنة على صرف أخر علاوة ، وتحتسب أول علاوة بعد مرور عام على استلام العامل للعمل . وقد قرر المشرع في قانون العمل مبدأ عام ، حيث جعل العلاوة الدورية السنوية جزءاً من الأجر وتأخذ حكمه ( المادة رقم 1 فقرة ج بند 3 ) . وبهذا المعنى تعتبر العلاوة الدورية السنـوية زيادة سنـوية تعطى للعـامل زيـادة على أجره الأصلي ( أجر الالتحاق بالعمل ) . والعلاوة الدورية السنوية ، منشأها إرادة المشرع ، فهي مقررة بمقتضى حكم المادة الثانية من مواد إصدار قانون العمل الصادر بالقانون رقم 12 لسنه 2003 ، وهى لا تقل عن 7 % من الأجر الأساسي الذي تحسب على أساسه اشتراكات التأمينات الاجتماعية ( دون حد أدنى أو أقصى ) على خلاف ما كان مقرراً في قانون العمل السابق بمقتضى نص المادة 42 حين جعل المشرع حدها الأدنى جنيهان وحدها الأقصى سبعة جنيهات . ويلتزم أصحاب الأعمال – أيا كان عدد عمالهم – بصرف العلاوة الدورية السنوية في تاريخ استحقاقها ، وبنسبتها المقررة حسب التفصيل السابق و صدر الحكم التالي الذي جاء فيه

In [ ]:
#question = "ما هي الأعمال والمهن والصناعات التي لا يجوز تشغيل الأطفال فيها إذا قلت سنهم عن سبع عشرة سنة ؟"
#print(question)
#question_embed = model.encode(question)
#results = collection.query(
#    query_embeddings=question_embed.tolist(),
#    n_results=2
#)
#print(results)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# model_id = "silma-ai/SILMA-9B-Instruct-v1.0"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# llm_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
# )

model_id = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)


In [ ]:
from langchain import PromptTemplate

# Creating the prompt template
qna_template = "\n".join([
    "Answer the next question using the provided context.",
    "If the answer is not contained in the context, say 'NO ANSWER IS AVAILABLE'.",
    "Provide a detailed and thorough explanation, explaining the context and including any relevant legal or procedural details if necessary.",
    "### Context:",
    "{context}",
    "",
    "### Question:",
    "{question}",
    "",
    "### Answer:",
])




qna_prompt = PromptTemplate(
    template=qna_template,
    input_variables=['context', 'question'],
    verbose=True
)

# Retreiving from the vector database
question = questions[6]
question_embed = model.encode(question)

results = collection.query(
    query_embeddings=question_embed.tolist(),
    n_results=3
)
context = results["documents"][0][0]

# Formating the prompt with the variables
prompt = qna_prompt.format(context=context, question=question)


# Generating
messages = [
    {"role": "system", "content": "أنت مساعد ذكي للإجابة عن أسئلة المستخدمين."},
    {"role": "user", "content": prompt},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = llm_model.generate(**input_ids, max_new_tokens=256)

print(tokenizer.decode(outputs[0]))

<bos>أنت محامي ذكي للإجابة عن أسئلة المستخدمين.<start_of_turn>user
Answer the next question using the provided context.
If the answer is not contained in the context, say 'NO ANSWER IS AVAILABLE'.
Provide a detailed and thorough explanation, explaining the context and including any relevant legal or procedural details if necessary.
### Context:
ﺗﻛﻭﻥ ﻣﻘﺭﺭﺓ ﺃﻭ ﺗﻘﺭﺭ ﻓﻲ ﻋﻘﻭﺩ ﺍﻟﻌﻣﻝ ﺍﻟﻔﺭﺩﻳﺔ ﺃﻭ ﺍﻟﺟﻣﺎﻋﻳﺔ ﺃﻭ ﺍﻷﻧﻅﻣﺔ ﺍﻷﺳﺎﺳﻳﺔ ﺃﻭ ﻏﻳﺭﻫﺎ ﻣﻥ ﻟﻭﺍﺋﺢ ﺍﻟﻣﻧﺷﺄﺓ ﺃﻭ ﺑﻣﻘﺗﺿﻰ ﺍﻟﻌﺭﻑ ﻭﺗﻘﻊ ﺑﺎﻁﻠﺔ ﻛﻝ ﻣﺻﺎﻟﺣﺔ ﺗﺗﺿﻣﻥ ﺍﻧﺗﻘﺎﺻﺎ ﺃﻭ ﺇﺑﺭﺍء ﻣﻥ ﺣﻘﻭﻕ ﺍﻟﻌﺎﻣﻝ ﺍﻟﻧﺎﺷﺋﺔ ﻋﻥ ﻋﻘﺩ ﺍﻟﻌﻣﻝ ﺧﻼﻝ ﻣﺩﺓ ﺳﺭﻳﺎﻧﻪ ﺃﻭ ﺧﻼﻝ ﺛﻼﺛﺔ ﺃﺷﻬﺭ ﻣﻥ ﺗﺎﺭﻳﺦ ﺍﻧﺗﻬﺎﺋﻪ ﻣﺗﻰ ﻛﺎﻧﺕ ﺗﺧﺎﻟﻑ ﺃﺣﻛﺎﻡ ﻫﺫﺍ ﺍﻟﻘﺎﻧﻭﻥ ﻣﺎﺩﺓ6 ﺗﻌﻔﻰ ﻣﻥ ﺍﻟﺭﺳﻭﻡ

### Question:
ما هو الميعاد الخاص ببطلان أي مخالصة عن حق من حقوق العامل؟

### Answer:<end_of_turn>
<start_of_turn>model
ثلاثة أشهر من تاريخ انتهاء العقد<eos>


In [ ]:
answers[6]

'يقع باطلاً كل شرط أو اتفاق يخالف أحكام هذا القانون ولو كان سابقاً علي العمل به ، إذا كان يتضمن انتقاصاً من حقوق العامل المقررة فيه. ويستمر العمل بأية مزايا أو شروط تكون مقررة أو تقرر في عقود العمل الفردية أو الجماعية أو الأنظمة الأساسية أو غيرها من لوائح المنشأة ، أو بمقتضى العرف . وتقع باطلة كل مصالحة تتضمن انتقاصاً أو إبراء من حقوق العامل الناشئة عن عقد العمل خلال مدة سريانه أو خلال ثلاثة أشهر من تاريخ انتهائه متي كانت تخالف أحكام هذا القانون'

In [ ]:
questions[3]

'س : ما حق صاحب العمل في رفضه قيام العامل بالاجازة السنوية ؟'

In [ ]:
question = questions[3]
question_embed = model.encode(question)

results = collection.query(
    query_embeddings=question_embed.tolist(),
    n_results=3
)
context = results["documents"][0][0]

# Formating the prompt with the variables
prompt = qna_prompt.format(context=context, question=question)


# Generating
messages = [
    {"role": "system", "content": "أنت مساعد ذكي للإجابة عن أسئلة المستخدمين."},
    {"role": "user", "content": prompt},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = llm_model.generate(**input_ids, max_new_tokens=256)

print(tokenizer.decode(outputs[0]))

<bos>أنت مساعد ذكي للإجابة عن أسئلة المستخدمين.<start_of_turn>user
Answer the next question using the provided context.
If the answer is not contained in the context, say 'NO ANSWER IS AVAILABLE'.
Provide a detailed and thorough explanation, explaining the context and including any relevant legal or procedural details if necessary.
### Context:
ﺻﺎﺣﺏ ﺍﻟﻌﻣﻝ ﻭﺇﺫﺍ ﺭﻓﺽ ﺍﻟﻌﺎﻣﻝ ﻛﺗﺎﺑﺔ ﺍﻟﻘﻳﺎﻡ ﺑﺎﻹﺟﺎﺯﺓ ﺳﻘﻁ ﺣﻘﻪ ﻓﻰ ﺍﻗﺗﺿﺎء ﻣﻘﺎﺑﻠﻬﺎ ﻭﻓﻰ ﺟﻣﻳﻊ ﺍﻷﺣﻭﺍﻝ ﻳﺟﺏ ﺃﻥ ﻳﺣﺻﻝ ﺍﻟﻌﺎﻣﻝ ﻋﻠﻰ ﺇﺟﺎﺯﺓ ﺳﻧﻭﻳﺔ ﻣﺩﺗﻬﺎ ﺧﻣﺳﺔ ﻋﺷﺭ ﻳﻭﻣﺎ ﻣﻧﻬﺎ ﺳﺗﺔ ﺃﻳﺎﻡ ﻣﺗﺻﻠﺔ ﻋﻠﻰ ﺍﻷﻗﻝ ﻭﻳﻠﺗﺯﻡ ﺻﺎﺣﺏ ﺍﻟﻌﻣﻝ ﺑﺗﺳﻭﻳﺔ ﺭﺻﻳﺩ ﺍﻹﺟﺎﺯﺍﺕ ﺃﻭ ﺍﻷﺟﺭ ﺍﻟﻣﻘﺎﺑﻝ ﻟﻪ ﻛﻝ ﺛﻼﺙ ﺳﻧﻭﺍﺕ ﻋﻠﻰ ﺍﻷﻛﺛﺭ ﻓﺈﺫﺍ ﺍﻧﺗﻬﺕ ﻋﻼﻗﺔ ﺍﻟﻌﻣﻝ ﻗﺑﻝ ﺍﺳﺗﻧﻔﺎﺩ ﺍﻟﻌﺎﻣﻝ ﺭﺻﻳﺩ

### Question:
س : ما حق صاحب العمل في رفضه قيام العامل بالاجازة السنوية ؟

### Answer:<end_of_turn>
<start_of_turn>model
لا<eos>


In [ ]:
answers[3]

'ج: يبحث هذا الموضوع وإعمالاً لأحكام العمل الصادر برقم 12 لسنه 2003 المادة 48 منه ، فيحدد صاحب العمل مواعيد الإجازة السنوية حسب مقتضيات العمل وظروفه ولا يجوز قطعها إلا لأسباب قوية تقتضيها مصلحة العمل ، ويلتزم العامل بالقيام بالإجازة في التاريخ والمدة التى حددها صاحب العمل وإذا رفض العامل كتابتاً القيام بالإجازة سقط حقه في اقتضاء مقابلها ، وفي جميع الأحوال يجب أن يحصل العامل على إجازة سنوية مدتها خمسة عشر يوماً منها ستة أيام متصلة على الأقل ويلتزم صاحب العمل بتسوية رصيد الأجازات أو الأجر المقابل له كثلاث سنوات على الأكثر فإذا انتهت علاقة العمل قبل استنفاد العامل رصيد إجازته السنوية استحق الأجر المقابل لهذا الرصيد ، وللعامل طبقاً لأحكام المادة 49 الحق في تحديد موعد إجازته السنوية إذا كان متقدماً لأداء الامتحان في إحدى مراحل التعليم بشرط أن يخطر صاحب العمل قبل قيامه ب؟لإجازة بخمسة عشر يوماً على الأقل ، مما سبق يتضح أن المشرع في قانون العمل الجديد قد أبقى على حق صاحب العمل في تحديد مواعيد إجازة العامل وتجزئتها ولكن في حدود ما قرره من أحكام لمدة الإجازة وفقاً للأقدمية أو السن والقواعد المتع

In [ ]:
# Assuming `questions` is a list or array with your questions
question_7 = questions[10]

# Generate the embedding for question[7]
question_7_embed = model.encode(question_7)

# Retrieve the context from the vector database for question[7]
results_7 = collection.query(
    query_embeddings=question_7_embed.tolist(),
    n_results=3
)

# Extract the context (the most relevant document)
context_7 = results_7["documents"][0][0]

# Format the prompt with the context and question[7]
prompt_7 = qna_prompt.format(context=context_7, question=question_7)

# Generate the response from the model
messages = [
    {"role": "system", "content": "أنت مساعد ذكي للإجابة عن أسئلة المستخدمين."},
    {"role": "user", "content": prompt_7},
]

# Apply the chat template and generate the response
input_ids_7 = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

# Generate the output for question[7]
outputs_7 = llm_model.generate(**input_ids_7, max_new_tokens=256)

# Decode and print the answer
answer_7 = tokenizer.decode(outputs_7[0])
print(answer_7)

<bos>أنت مساعد ذكي للإجابة عن أسئلة المستخدمين.<start_of_turn>user
Answer the next question using the provided context.
If the answer is not contained in the context, say 'NO ANSWER IS AVAILABLE'.
Provide a detailed and thorough explanation, explaining the context and including any relevant legal or procedural details if necessary.
### Context:
ﺩﻭﻥ ﺍﻟﺣﺻﻭﻝ ﻋﻠﻰ ﺍﻟﺗﺭﺧﻳﺹ ﺍﻟﻣﻧﺻﻭﺹ ﻋﻠﻳﻪ ﻓﻳﻬﺎ ﺃﻭ ﺑﺗﺭﺧﻳﺹ ﺻﺎﺩﺭ ﺑﻧﺎء ﻋﻠﻰ ﺑﻳﺎﻧﺎﺕ ﻏﻳﺭ ﺻﺣﻳﺣﺔ ﺗﻘﺎﺿﻰ ﻣﺑﺎﻟﻎ ﻣﻥ ﺍﻟﻌﺎﻣﻝ ﻧﻅﻳﺭ ﺇﻟﺣﺎﻗﻪ ﺑﺎﻟﻌﻣﻝ ﺩﺍﺧﻝ ﺟﻣﻬﻭﺭﻳﺔ ﻣﺻﺭ ﺍﻟﻌﺭﺑﻳﺔ ﺃﻭ ﺧﺎﺭﺟﻬﺎ ﺑﺎﻟﻣﺧﺎﻟﻔﺔ ﻷﺣﻛﺎﻡ ﺍﻟﻣﺎﺩﺓ 21 ﻣﻥ ﻫﺫﺍ ﺍﻟﻘﺎﻧﻭﻥ ﺃﻭ ﺗﻘﺎﺿﻰ ﻣﺑﺎﻟﻎ ﺩﻭﻥ ﻭﺟﻪ ﺣﻕ ﻣﻥ ﺃﺟﺭ ﺍﻟﻌﺎﻣﻝ ﺃﻭ ﻣﻥ ﻣﺳﺗﺣﻘﺎﺗﻪ ﻋﻥ ﻋﻣﻠﻪ ﻓﻰ ﺍﻟﺩﺍﺧﻝ ﺃﻭ ﻓﻰ ﺍﻟﺧﺎﺭﺝ ﻣﺧﺎﻟﻔﺔ ﺍﻷﺣﻛﺎﻡ

### Question:
ما هي حالات إلغاء وزير القوي العاملة للترخيص الممنوح للشركات بالعمل في مجال إلحاق الأيدي العاملة المصرية للعمل بالخارج والداخل ؟

### Answer:<end_of_turn>
<start_of_turn>model
إذا كان الترخيص قد صدر بناءً على بيانات غير صحيحة<eos>


In [ ]:
answers[10]

'- يلغي الترخيص بقرار من الوزير المختص عند ثبوت أي حالة من الحالات الآتية: فقد الشركة شرطاً من شروط الترخيص. تقاضي الشركة أية مبالغ من العامل نظير تشغيله بالمخالفة لأحكام هذا الفصل. حصول الشركة علي الترخيص او تجديده أو عدم اعتراض الوزارة علي اتفاق او عقد عمل بناء علي ما قدمته من بيانات غير صحيحة . ويجوز بقرار من الوزير المختص إلغاء الترخيص في حالة ثبوت مخالفة الشركة لحكم من الأحكام الجوهرية الواردة في القرارات الصادرة تنفيذاً لأحكام هذا الفصل . وللوزير المختص إيقاف نشاط الشركة مؤقتاً إذا نسب إليها بناء علي أسباب جدية أي من الحالات المبينة في هذه النادرة ، وذلك لحين الفصل في مدي ثبوت تلك الحالات أو حين زوال المخالفة في الحالة المنصوص عليها في الفقرة السابقة . ولا يخل إلغاء الترخيص في أي من الحالات المبيتة في هذه المادة بالمسئولية الجنائية أو المدنية أو التأديبية .'

In [ ]:
# Assuming `questions` is a list or array with your questions
question_7 = questions[10]

# Generate the embedding for question[7]
question_7_embed = model.encode(question_7)

# Retrieve the context from the vector database for question[7]
results_7 = collection.query(
    query_embeddings=question_7_embed.tolist(),
    n_results=5
)

# Extract the context (the most relevant document)
context_7 = results_7["documents"][0][0]

# Format the prompt with the context and question[7]
prompt_7 = qna_prompt.format(context=context_7, question=question_7)

# Generate the response from the model
messages = [
    {"role": "system", "content": "أنت مساعد ذكي للإجابة عن أسئلة المستخدمين."},
    {"role": "user", "content": prompt_7},
]

# Apply the chat template and generate the response
input_ids_7 = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

# Generate the output for question[7]
outputs_7 = llm_model.generate(**input_ids_7, max_new_tokens=256)

# Decode and print the answer
answer_7 = tokenizer.decode(outputs_7[0])
print(answer_7)

<bos>أنت مساعد ذكي للإجابة عن أسئلة المستخدمين.<start_of_turn>user
Answer the next question using the provided context.
If the answer is not contained in the context, say 'NO ANSWER IS AVAILABLE'.
Provide a detailed and thorough explanation, explaining the context and including any relevant legal or procedural details if necessary.
### Context:
ﺩﻭﻥ ﺍﻟﺣﺻﻭﻝ ﻋﻠﻰ ﺍﻟﺗﺭﺧﻳﺹ ﺍﻟﻣﻧﺻﻭﺹ ﻋﻠﻳﻪ ﻓﻳﻬﺎ ﺃﻭ ﺑﺗﺭﺧﻳﺹ ﺻﺎﺩﺭ ﺑﻧﺎء ﻋﻠﻰ ﺑﻳﺎﻧﺎﺕ ﻏﻳﺭ ﺻﺣﻳﺣﺔ ﺗﻘﺎﺿﻰ ﻣﺑﺎﻟﻎ ﻣﻥ ﺍﻟﻌﺎﻣﻝ ﻧﻅﻳﺭ ﺇﻟﺣﺎﻗﻪ ﺑﺎﻟﻌﻣﻝ ﺩﺍﺧﻝ ﺟﻣﻬﻭﺭﻳﺔ ﻣﺻﺭ ﺍﻟﻌﺭﺑﻳﺔ ﺃﻭ ﺧﺎﺭﺟﻬﺎ ﺑﺎﻟﻣﺧﺎﻟﻔﺔ ﻷﺣﻛﺎﻡ ﺍﻟﻣﺎﺩﺓ 21 ﻣﻥ ﻫﺫﺍ ﺍﻟﻘﺎﻧﻭﻥ ﺃﻭ ﺗﻘﺎﺿﻰ ﻣﺑﺎﻟﻎ ﺩﻭﻥ ﻭﺟﻪ ﺣﻕ ﻣﻥ ﺃﺟﺭ ﺍﻟﻌﺎﻣﻝ ﺃﻭ ﻣﻥ ﻣﺳﺗﺣﻘﺎﺗﻪ ﻋﻥ ﻋﻣﻠﻪ ﻓﻰ ﺍﻟﺩﺍﺧﻝ ﺃﻭ ﻓﻰ ﺍﻟﺧﺎﺭﺝ ﻣﺧﺎﻟﻔﺔ ﺍﻷﺣﻛﺎﻡ

### Question:
ما هي حالات إلغاء وزير القوي العاملة للترخيص الممنوح للشركات بالعمل في مجال إلحاق الأيدي العاملة المصرية للعمل بالخارج والداخل ؟

### Answer:<end_of_turn>
<start_of_turn>model
إذا كان الترخيص قد صدر بناءً على بيانات غير صحيحة<eos>


In [ ]:
"""
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample answers
answer1 = answer_7
answer2 = answers[10]

# Convert text to TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([answer1, answer2])

# Calculate cosine similarity
similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

print(f"Cosine Similarity: {similarity[0][0]}")
"""

Cosine Similarity: 0.03965624085274666


# Bleu Score

In [ ]:
"""
from nltk.translate.bleu_score import corpus_bleu

# Create a list to store the model's answers
model_answers = []

# Assuming 'questions' and 'answers' are already defined, and 'llm_model' and 'tokenizer' are your language model and tokenizer
for i, question in enumerate(questions):
    # Formatting the prompt with the current question and context
    prompt = qna_prompt.format(context=context, question=question)

    # Prepare messages for the model
    messages = [
        {"role": "system", "content": "أنت مساعد ذكي للإجابة عن أسئلة المستخدمين."},
        {"role": "user", "content": prompt},
    ]

    # Tokenize the input and prepare it for the model
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

    # Generate the model's answer
    outputs = llm_model.generate(**input_ids, max_new_tokens=256)

    # Decode the model's answer
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Append the model's answer to the list
    model_answers.append(model_answer)

# Now you can calculate BLEU score
references = []
candidates = []

# Iterate through all the similarities to collect references and candidates
for i, model_answer in enumerate(model_answers):
    # Get the real answer (reference)
    real_answer = answers[i]

    # Tokenize the real (reference) answer
    references.append([real_answer.split()])

    # Tokenize the model (candidate) answer
    candidates.append(model_answer.split())

# Calculate the BLEU score for the entire corpus
bleu_score = corpus_bleu(references, candidates)

# Print the final BLEU score
print(f"Final BLEU score: {bleu_score:.4f}")
"""

# Streamlit Deployment


In [ ]:
!pip install -q pyngrok
!pip install -q streamlit

In [ ]:
from pyngrok import ngrok, conf
import os
import subprocess
import streamlit
import getpass


# Ensure you have all the required files
if not os.path.exists("/content/app.py"):
    raise FileNotFoundError("The app.py file is not found in the current directory.")

# Run the Streamlit app as a subprocess
process = subprocess.Popen(["streamlit", "run", "/content/app.py", "--server.port", "8501"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=8501
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

# Stream the subprocess output to check for issues
try:
    for line in process.stdout:
        print(line.decode("utf-8").strip())
except KeyboardInterrupt:
    print("Stopping Streamlit app...")
    process.terminate()
    ngrok.disconnect(public_url)

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel "https://9b73-34-143-165-69.ngrok-free.app" -> "http://127.0.0.1:8501"



You can now view your Streamlit app in your browser.

Local URL: http://localhost:8501
Network URL: http://172.28.0.12:8501
External URL: http://34.143.165.69:8501

